In [2]:
import numpy as np
import torch
import itertools

In [4]:
b=200
baggage = {"A":[100,60],"B":[80,30],"C":[50,10],"D":[220,160],"E":[70,20],"F":[50,100],"G":[80,20],
            "H":[60,30],"I":[40,40],"J":[90,30],"K":[100,30],"L":[50,50],"M":[80,160],"N":[80,50]}

In [3]:
sample_data = ["A","B","C"]

In [9]:
print(list(itertools.combinations(baggage,2)))
print(len(list(itertools.combinations(baggage,2))))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('A', 'F'), ('A', 'G'), ('A', 'H'), ('A', 'I'), ('A', 'J'), ('A', 'K'), ('A', 'L'), ('A', 'M'), ('A', 'N'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('B', 'F'), ('B', 'G'), ('B', 'H'), ('B', 'I'), ('B', 'J'), ('B', 'K'), ('B', 'L'), ('B', 'M'), ('B', 'N'), ('C', 'D'), ('C', 'E'), ('C', 'F'), ('C', 'G'), ('C', 'H'), ('C', 'I'), ('C', 'J'), ('C', 'K'), ('C', 'L'), ('C', 'M'), ('C', 'N'), ('D', 'E'), ('D', 'F'), ('D', 'G'), ('D', 'H'), ('D', 'I'), ('D', 'J'), ('D', 'K'), ('D', 'L'), ('D', 'M'), ('D', 'N'), ('E', 'F'), ('E', 'G'), ('E', 'H'), ('E', 'I'), ('E', 'J'), ('E', 'K'), ('E', 'L'), ('E', 'M'), ('E', 'N'), ('F', 'G'), ('F', 'H'), ('F', 'I'), ('F', 'J'), ('F', 'K'), ('F', 'L'), ('F', 'M'), ('F', 'N'), ('G', 'H'), ('G', 'I'), ('G', 'J'), ('G', 'K'), ('G', 'L'), ('G', 'M'), ('G', 'N'), ('H', 'I'), ('H', 'J'), ('H', 'K'), ('H', 'L'), ('H', 'M'), ('H', 'N'), ('I', 'J'), ('I', 'K'), ('I', 'L'), ('I', 'M'), ('I', 'N'), ('J', 'K'), ('J', 'L'), ('J

In [23]:
def comb_num(n,r):
    n_list=[i for i in range(n,0,-1)]
    cnum=0
    up=1
    down=1
    up_list=n_list[:r]
    down_list=n_list[-r:]
    for i in up_list:
        up *= i
    for j in down_list:
        down *= j
    cnum=up/down
    print(n_list,up,down,cnum)
comb_num(6,2)


[6, 5, 4, 3, 2, 1] 30 2 15.0


In [13]:
[i for i in range(6,0,-1)]

[6, 5, 4, 3, 2, 1]